In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import spacy
import dataset
import analyze

In [29]:
!python -m spacy download it_core_news_sm

/bin/bash: python: command not found


In [3]:
# likes_path = "easytour-likes-2022-03MAR-21.json"
# schede_path = "easytour-schede-2022-03MAR-21.json"
#likes_path = "kuriu-likes-apr.json"
#schede_path = "kuriu-schede-apr.json"
likes_path = "kuriu-likes-2022-06JUN-18.json"
schede_path = "kuriu-schede-2022-06JUN-18.json"
data_path = "data-JUN.csv"
data_analyzed_path = "data_analyzed-JUN.csv"

In [4]:
f_schede = open(schede_path)
data_schede = json.load(f_schede)
print("Fields: {}".format(data_schede[0].keys())) 
print("Number of samples: {}".format(len(data_schede)))

Fields: dict_keys(['id', 'userId', 'userName', 'documentType', 'title', 'description', 'locations', 'duration', 'imageId', 'previewId', 'distance', 'datePublishing', 'widgets', 'counters', 'features', 'tags', 'numberOfPositions', 'categories', 'videoId', 'creationDate', 'version', 'media', 'shareType', 'status', 'lastUpdate', 'versionHistory', 'viewCounter', 'device', 'crowdPosting', 'activeCrowdPosting'])
Number of samples: 5724


In [5]:
print("First sample: {}".format(data_schede[8]))

First sample: {'id': '621128704f9d177c90f36d8a', 'userId': 579, 'userName': 'laviadellescimmie', 'documentType': 'experience', 'title': 'The Lion Rock, l’antica città di Sigiriya', 'description': 'Una delle attrazioni più suggestive di tutto lo Sri Lanka è la Lion Rock. Questa si trova all’interno di un parco chiamato Sigiriya. Si tratta di un’enorme roccia che svetta sulla vallata, immersa nella vegetazione. Si parla anche dell’antica città di Sigiriya perché sulla sua sommità si possono ancora ammirare le rovine di un antico palazzo (della fine del V secolo) e le zone coltivate. Sigiriya rientra nei patrimoni dell’umanità dello Sri Lanka. \n\nPer evitare di imbattersi in un gran numero di turisti, è meglio visitare il sito archeologico alla mattina, all’apertura. La luce è migliore, il clima è clemente e ci sono meno persone. La roccia deriva dall’erosione di un antico vulcano, ecco perché la sua forma è così particolare e unica in tutto il Paese. La roccia si trova su una collina al

In [6]:
f_likes = open(likes_path)
data_likes = json.load(f_likes)
print("Number of samples: {}".format(len(data_likes)))
print("Fields: {}".format(data_likes[0].keys()))
print("First sample: {}".format(data_likes[0]))

Number of samples: 2542
Fields: dict_keys(['documentId', 'lastUpdate', 'authorId', 'userName', 'type'])
First sample: {'documentId': '622c3b8ac992ca28f33c7f21', 'lastUpdate': '2022-03-19T07:56:20.868Z', 'authorId': 606, 'userName': 'DaleOnTravel', 'type': 'Useful'}


In [7]:
df_schede = pd.read_json(schede_path)
df_likes = pd.read_json(likes_path)

<h3>Data Analysis</h3>

In [8]:
df_schede.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5724 entries, 0 to 5723
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  5724 non-null   object 
 1   userId              5724 non-null   int64  
 2   userName            5724 non-null   object 
 3   documentType        5724 non-null   object 
 4   title               5724 non-null   object 
 5   description         5724 non-null   object 
 6   locations           5724 non-null   object 
 7   duration            5724 non-null   int64  
 8   imageId             5723 non-null   object 
 9   previewId           598 non-null    object 
 10  distance            5724 non-null   int64  
 11  datePublishing      5713 non-null   object 
 12  widgets             4788 non-null   object 
 13  counters            5724 non-null   object 
 14  features            5724 non-null   object 
 15  tags                5724 non-null   object 
 16  number

In [9]:
df_schede.describe()

,userId,duration,distance,numberOfPositions,version,viewCounter,crowdPosting
count,5724.000000,5.724000e+03,5724.0,5724.000000,5724.000000,5724.000000,0.0
mean,458.611286,1.848888e+07,0.0,1.300314,6.140985,41.306953,NaN
std,151.017776,6.209633e+07,0.0,1.000328,9.479419,42.832072,NaN
min,77.000000,0.000000e+00,0.0,1.000000,2.000000,1.000000,NaN
25%,314.000000,0.000000e+00,0.0,1.000000,3.000000,15.000000,NaN
50%,498.000000,3.600000e+06,0.0,1.000000,4.000000,45.000000,NaN
75%,572.500000,1.080000e+07,0.0,1.000000,7.000000,59.000000,NaN
max,708.000000,1.296000e+09,0.0,12.000000,414.000000,2134.000000,NaN


In [10]:
df_likes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   documentId  2542 non-null   object
 1   lastUpdate  2542 non-null   object
 2   authorId    2542 non-null   int64 
 3   userName    2542 non-null   object
 4   type        2542 non-null   object
dtypes: int64(1), object(4)
memory usage: 99.4+ KB


In [11]:
#df_schede_fields = df_schede[["id","userId", "title", "description", "duration", "datePublishing", "creationDate", "counterUseful", "categories", "viewCounter"]]
df_schede_fields = df_schede[["id","userId", "title", "description", "duration", "datePublishing", "creationDate", "categories", "viewCounter"]]
#df_likes_ids = df_likes[["id", "userId"]]
df_likes_ids = df_likes[["documentId", "authorId"]]

In [12]:
df_schede_fields

,id,userId,title,description,duration,datePublishing,creationDate,categories,viewCounter
0,62349ff2272ae5672fbc3135,632,"Big Buddha, Phuket 🇹🇭",Phuket è conosciuta per essere la meta predile...,0,2022-03-19T04:48:49.248Z,2022-03-18T15:06:26.000+00:00,[Viaggi],18
1,6234aa65272ae5672fbc34c2,632,"Wat Chalong, il tempio più venerato di Phuket 🇹🇭","Phuket non è solo spiagge, ci sono anche altri...",0,2022-03-19T04:04:27.356Z,2022-03-18T15:51:01.000+00:00,[Viaggi],16
2,620aa536ff8ae67f37cafb06,442,Accarezzare le tigri,So che dietro queste cose c’è spesso del marci...,10800000,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,[Viaggi],426
3,624e5982ec9ff859ff529ce8,571,"Yangon, la capitale storica del Myanmar",Probabilmente Yangon sarà il punto d’inizio se...,0,2022-04-07T03:24:51.101Z,2022-04-07T03:24:50.000+00:00,[Viaggi],6
4,620abb33ff8ae67f37cafe3c,442,Freedom beach,Forse la spiaggia più bella di tutta l’isola d...,18000000,2022-02-14T20:27:32.224Z,2022-02-14T20:27:31.000+00:00,[Viaggi],424
...,...,...,...,...,...,...,...,...,...
5719,6213b3460f41f86438b07d9c,429,"Machu Picchu, meraviglia del mondo","È l’alba, ma la realtà è che in questo viaggio...",21600000,2022-02-21T15:44:06.580Z,2022-02-21T15:44:06.000+00:00,[Viaggi],47
5720,61dd867f9c823e129b1f2cc2,429,I segreti di Moray,Sembra la prova schiacciante dell’esistenza de...,7200000,2022-01-11T13:30:45.997Z,2022-01-11T13:30:39.000+00:00,[Aria aperta],95
5721,61e0146111b3bf4d3185ad5d,427,"Machu Picchu, tra storia e meraviglia",Sono passati dieci (!) anni dalla mia esperien...,16200000,2022-01-13T12:04:35.591Z,2022-01-13T12:00:33.000+00:00,[Cultura],63
5722,6273a0a9dd67f6198e5ac192,606,Come arrivare a Machu Picchu: modalità e tempi,Sembrerà un argomento banale: vuoi che sia dif...,0,2022-05-05T10:02:17.841Z,2022-05-05T10:02:17.000+00:00,[Viaggi],7


<h3>Preprocessing</h3>

<h5>Null or empty values</h5>

In [13]:
#Check if there are null fields
df_schede_fields.isnull().sum()

id                 0
userId             0
title              0
description        0
duration           0
datePublishing    11
creationDate       0
categories         1
viewCounter        0
dtype: int64

In [14]:
def fill_null_fields(df):
    #empty description -> remove or use '' ?
    df["description"].fillna('', inplace=True)
    #if duration is null -> set to 0
    df["duration"].fillna(0, inplace=True)
    #if datePublishing is null -> set it equal to creationDate
    df["datePublishing"].fillna(df_schede_fields.creationDate, inplace=True)
    #if viewCounter is null -> set to 0
    df["viewCounter"].fillna(0, inplace=True)

In [15]:
fill_null_fields(df_schede_fields)

/tmp/ipykernel_161862/1698813668.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["description"].fillna('', inplace=True)
/tmp/ipykernel_161862/1698813668.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["duration"].fillna(0, inplace=True)
/tmp/ipykernel_161862/1698813668.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datePublishing"].fillna(df_schede_fields.creationDate, inplace=True)
/tmp/ipykernel_161862/1698813

<h5>Categories into one hot encoding</h5>

In [16]:
def lower_list(l):
    if type(l)!=list:
        return []
    res = [x.lower() for x in l]
    return res

def encoding_categories(df):
  one_hot_enc_categories = df["categories"].apply(lambda x: lower_list(x))
  df["categories_encoding"] = one_hot_enc_categories
  mlb = MultiLabelBinarizer()

  schede_with_categories = df.join(
              pd.DataFrame(
                  mlb.fit_transform(df.pop('categories_encoding')),
                  index=df.index,
                  columns=mlb.classes_))
  return schede_with_categories

In [17]:
df_categories = encoding_categories(df_schede_fields)

/tmp/ipykernel_161862/3170635180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["categories_encoding"] = one_hot_enc_categories


<h5>Text fields</h5>

In [18]:
import string
from bs4 import BeautifulSoup
import nltk
import unidecode
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [19]:
#Downloads for stopwords and punctuation
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('italian')
nltk.download('wordnet')
string.punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gianfree/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gianfree/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [20]:
def remove_html(text: string):
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text(separator=" ")
    return cleaned_text

In [21]:
def remove_accented_chars(text: string):
    text = unidecode.unidecode(text)
    return text

In [22]:
def remove_stopwords(text: string):
  output= [i for i in text.split() if i not in stopwords]
  return output

In [23]:
#Removing punctuation
def remove_punctuation(text: string):
  list_without_punctuation = []
  for i in text:
    if i not in string.punctuation:
      list_without_punctuation.append(i)
    else:
      list_without_punctuation.append(' ')
  string_without_punctuation = ''.join(list_without_punctuation)
  return string_without_punctuation

In [24]:
#Removing numbers
def remove_numbers(text: string):
  list_without_numbers = []
  for i in text:
    if i not in '0123456789':
      list_without_numbers.append(i)
    else:
      list_without_numbers.append(' ')
  string_without_numbers = ''.join(list_without_numbers)
  return string_without_numbers

In [25]:
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatizer(text: string):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text

In [26]:
#Stemming
italian_stemmer = SnowballStemmer('italian')

def stemming(text: string):
  stem_text = [italian_stemmer.stem(word) for word in text if not word.isdigit()]
  return stem_text

In [27]:
def preprocessing(text: string, html=1, accent=0, punct=1, numb=1, stop=1, lemma=0, stem=0):
    if html==1:
        text = remove_html(text)
    if accent==1:
        text = remove_accented_chars(text)
    if punct==1:
        text = remove_punctuation(text)
    if numb==1:
        text = remove_numbers(text)
    if stop==1:
        text = remove_stopwords(text)
    if lemma==1:
        text = lemmatizer(text)
    if stem==1:
        text = stemming(text)
    return text

In [28]:
nlp = spacy.load("it_core_news_sm")

def keep_just_nouns(field):
    doc = nlp(field.lower())
    return [token.lemma_ for token in doc if token.pos_=="NOUN"]

def remove_verbs_adj(field):
    doc = nlp(field.lower())
    return [token.lemma_ for token in doc if token.pos_ not in ["VERB", "ADJ", "ADV"]]

OSError: [E050] Can't find model 'it_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
def clean_text(df):
    df["title"] = df["title"].apply(lambda x: analyze.demojize(x))
    df["description"] = df["description"].apply(lambda x: analyze.demojize(x))
    # Remove Verbs from description
    schede_description = df.apply(
        lambda row: remove_verbs_adj(row['description']),
        axis=1)

    # Remove hastags, punctuation, stop words and numbers
    schede_description = schede_description.apply(
        lambda row: dataset.clean_text_aslist(row)) 
        
    # Remove Verbs from title
    schede_title = df.apply(
        lambda row: remove_verbs_adj(row['title']),
        axis=1)
        
    # Remove hastags, punctuation, stop words and numbers
    schede_title = schede_title.apply(
        lambda row: dataset.clean_text_aslist(row))
    
    df["description"] = schede_description
    df["title"] = schede_title

In [ ]:
clean_text(prova_df)

ValueError: Length of values (3) does not match length of index (2)

In [ ]:
prova_df

<h5>Dates</h5>

In [ ]:
def preprocess_dates(df):
    # Convert duration from ms to minutes
    df['duration_min'] = df.apply(lambda row: row["duration"] / 60000, axis=1)

    # Compute age of entry in days from today
    df['days'] = 0
    df.days = (pd.to_datetime("now").tz_localize('UTC') - pd.to_datetime(
        df["creationDate"])) // np.timedelta64(1, 'D')

In [ ]:
preprocess_dates(df_categories)

/home/gianfree/.local/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


In [ ]:
df_categories

,id,userId,title,description,duration,datePublishing,creationDate,counterUseful,categories,viewCounter,aria aperta,cultura,food,sport,tempo libero,viaggi,duration_min,days
0,62355f98272ae5672fbc3ac7,632,"[spiaggia, phuket, thailand]","[phuket, paradiso, amant, sole, mare, sole, co...",0,2022-03-19T04:44:09.503Z,2022-03-19T04:44:08.000+00:00,0,[Viaggi],7,0,0,0,0,0,1,0.0,31
1,620aa536ff8ae67f37cafb06,442,[tigr],"[marcio, maltrattamento, animal, andar, thaila...",10800000,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,0,[Viaggi],419,0,0,0,0,0,1,180.0,63
2,622c3b8ac992ca28f33c7f21,571,"[escursion, tramonto, pidurangala, rock]","[cuor, sri, lanka, zona, interess, \n\r\n, sig...",0,2022-03-12T06:19:55.882Z,2022-03-12T06:19:54.000+00:00,1,[Aria aperta],24,1,0,0,0,0,0,0.0,38
3,620a90c8ff8ae67f37caf97f,530,"[koh, samui]","[tour, bangkok, arcipelago, koh, samui, nott, ...",3600000,2022-02-16T15:15:51.387Z,2022-02-14T17:26:32.000+00:00,0,[Viaggi],453,0,0,0,0,0,1,60.0,63
4,622c398cc992ca28f33c7edd,571,"[visita, sito, dambulla, sri, lanka]","[sri, lanka, poter, mare, passeggiata, piantag...",0,2022-03-12T06:11:25.746Z,2022-03-12T06:11:24.000+00:00,1,[Viaggi],12,0,0,0,0,0,1,0.0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4642,620c32db25398565dadb5867,570,"[san, pedro, san, rafael, san, antonio, colombia]","[san, pedro, de, lo, milagro, nord, medellín, ...",194400000,2022-03-02T16:28:47.197Z,2022-02-15T23:10:19.000+00:00,0,[Viaggi],39,0,0,0,0,0,1,3240.0,62
4643,6213b3460f41f86438b07d9c,429,"[machu, picchu, meraviglia]","[alba, realtà, viaggio, rumor, cascata, capell...",21600000,2022-02-21T15:44:06.580Z,2022-02-21T15:44:06.000+00:00,0,[Viaggi],41,0,0,0,0,0,1,360.0,56
4644,6207b9cdb1d0940f122580b7,570,"[color, guatapé, colombia]","[guatapé, esplosion, color, maniera, novemila,...",72000000,2022-03-02T16:27:30.221Z,2022-02-12T13:44:45.000+00:00,0,[Viaggi],52,0,0,0,0,0,1,1200.0,65
4645,61f6a3e3306802033034d576,536,"[avventura, ecuador, quilotoa, cotopaxi]","[viaggio, bordo, pulman, bussola, viaggiator, ...",172800000,2022-01-30T14:43:17.547Z,2022-01-30T14:42:43.000+00:00,1,[Viaggi],43,0,0,0,0,0,1,2880.0,78


In [ ]:
#df_categories.to_csv(data_path, index=False)

In [ ]:
data = pd.read_csv(data_path)  

In [ ]:
data

,id,userId,title,description,duration,datePublishing,creationDate,counterUseful,categories,viewCounter,aria aperta,cultura,food,sport,tempo libero,viaggi,duration_min,days
0,62355f98272ae5672fbc3ac7,632,"['spiaggia', 'phuket', 'thailand']","['phuket', 'paradiso', 'amant', 'sole', 'mare'...",0,2022-03-19T04:44:09.503Z,2022-03-19T04:44:08.000+00:00,0,['Viaggi'],7,0,0,0,0,0,1,0.0,31
1,620aa536ff8ae67f37cafb06,442,['tigr'],"['marcio', 'maltrattamento', 'animal', 'andar'...",10800000,2022-02-14T20:05:09.586Z,2022-02-14T18:53:42.000+00:00,0,['Viaggi'],419,0,0,0,0,0,1,180.0,63
2,622c3b8ac992ca28f33c7f21,571,"['escursion', 'tramonto', 'pidurangala', 'rock']","['cuor', 'sri', 'lanka', 'zona', 'interess', '...",0,2022-03-12T06:19:55.882Z,2022-03-12T06:19:54.000+00:00,1,['Aria aperta'],24,1,0,0,0,0,0,0.0,38
3,620a90c8ff8ae67f37caf97f,530,"['koh', 'samui']","['tour', 'bangkok', 'arcipelago', 'koh', 'samu...",3600000,2022-02-16T15:15:51.387Z,2022-02-14T17:26:32.000+00:00,0,['Viaggi'],453,0,0,0,0,0,1,60.0,63
4,622c398cc992ca28f33c7edd,571,"['visita', 'sito', 'dambulla', 'sri', 'lanka']","['sri', 'lanka', 'poter', 'mare', 'passeggiata...",0,2022-03-12T06:11:25.746Z,2022-03-12T06:11:24.000+00:00,1,['Viaggi'],12,0,0,0,0,0,1,0.0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4642,620c32db25398565dadb5867,570,"['san', 'pedro', 'san', 'rafael', 'san', 'anto...","['san', 'pedro', 'de', 'lo', 'milagro', 'nord'...",194400000,2022-03-02T16:28:47.197Z,2022-02-15T23:10:19.000+00:00,0,['Viaggi'],39,0,0,0,0,0,1,3240.0,62
4643,6213b3460f41f86438b07d9c,429,"['machu', 'picchu', 'meraviglia']","['alba', 'realtà', 'viaggio', 'rumor', 'cascat...",21600000,2022-02-21T15:44:06.580Z,2022-02-21T15:44:06.000+00:00,0,['Viaggi'],41,0,0,0,0,0,1,360.0,56
4644,6207b9cdb1d0940f122580b7,570,"['color', 'guatapé', 'colombia']","['guatapé', 'esplosion', 'color', 'maniera', '...",72000000,2022-03-02T16:27:30.221Z,2022-02-12T13:44:45.000+00:00,0,['Viaggi'],52,0,0,0,0,0,1,1200.0,65
4645,61f6a3e3306802033034d576,536,"['avventura', 'ecuador', 'quilotoa', 'cotopaxi']","['viaggio', 'bordo', 'pulman', 'bussola', 'via...",172800000,2022-01-30T14:43:17.547Z,2022-01-30T14:42:43.000+00:00,1,['Viaggi'],43,0,0,0,0,0,1,2880.0,78


<h3>Preprocess using functions in support</h3>

In [ ]:
#analyze.analyze_dataset(schede_path, data_analyzed_path, [])